### How to install this package
I made this file because I am pretty code illiterate and serves as a reminder 

1. Make sure you have anaconda installed on your machine
2. CLone the github repo https://github.com/kommanderpi/BRepNet.git
3. Use 'conda env create -f BRepNet_environment.yml' to isntall all dependencies
4. Jupyter notebook for VSCode or similar works well for the visualisation
5. Download the raw data using:
        cd /path/to/where_you_keep_data/
        curl https://fusion-360-gallery-dataset.s3-us-west-2.amazonaws.com/segmentation/s2.0.0/s2.0.0.zip -o s2.0.0.zip
        unzip s2.0.0.zip
6. Process the data locally using this code in Anaconda powershell(make sure you have activated the conda environment you created using the .yml file) (takes 2-3 hours depending on your device)
        python -m pipeline.quickstart --dataset_dir /path/to/where_you_keep_data/ --num_workers 10
        we set num_workers to 10 - seems to perform the best locally
7. Train the model (takes between 5 and 10 hours depending on your GPU):
        python -m train.train --dataset_file /path/to/where_you_keep_data/s2.0.0/processed/dataset.json \
        --dataset_dir /path/to/where_you_keep_data/s2.0.0/processed/ \
        --max_epochs 200 \
        --num_workers=10 \
        --gpus=1
8. Train.train will save the best version of your training model under:
        /logs/../../checkpoints/filename.ckpt
9. Alternatively use the pretrained model provided by Autodesk:
        ../example_files/pretrained_models/pretrained_s2.0.0_extended_step_uv_net_features_0816_183419.ckpt

10. You can now run the scripts under ../notebooks/filename.ipynb
        - BRepNET_Walkthrough.ipynb --> this notebook is a collection of tools and implementation of the segmentation model.

### Once we have cloned the repo, downloaded and processed the dataset and trained the model - we can take BRepNet for a test drive
We can also skip the training step and use the pretrained model provided in the repo.

## Generating and Displaying the segmentation results

Use a pre-trained BRepNet model, load a STEP model and see predicted segmentation results. You can run this file without training the model first - as Autodesk provides a pretrained model () 

This notebook should be initiated in the main BRepNet folder.  

Improtant notes:
- To use these tools on a custom dataset of STEP files: make sure to add a directory and STEP files you want to segment with the model to: ..\example_files\< FOLDER_NAME >
- You can also add your own files to: ..\example_files\step_examples if you want to sample from a larger data set
- Run this file in a Jupyter notebook: make sure you are using Python 3.7


Below we first check that we are in the correct dir and change to the root of the BRepNet repo if not.

In [1]:
import os
if os.path.isfile('../models/brepnet.py'):
    # We are in the notebooks directory.  Change to the root
    os.chdir('../')

In [2]:
from pathlib import Path

# This code allows you to evaluate a pre-trained model for all step files in a folder
from eval.evaluate_folder import evaluate_folder

# This viewer allows you to visualize the results
from visualization.jupyter_segmentation_viewer import JupyterSegmentationViewer

Use `eval/evaluate_folder.py` to evaluate your STEP model. We need to supply the script with the path to the step files, the feature standadization and a pretrained model.


In [3]:
# Create a path to your step files:
step_folder = Path("C:\BRep_Net\example_files\CustomStep")

# We will also need to know the feature standardization for the dataset used to train the model
# This is found in the dataset file created by pipeline/build_dataset_file.py or pipeline/quickstart.py
feature_standardization = Path("./example_files/feature_standardization/s2.0.0_step_all_features.json")

# Here is the path to a pretrained model:
pretrained_model = Path("./example_files/pretrained_models/pretrained_s2.0.0_extended_step_uv_net_features_0816_183419.ckpt")

# If you want to use your own model, you can uncomment the following line and change the path to your model:

# Here is a path to the model we trained ourselves:
# pretrained_model = Path("C:\BrepNet\BRepNet\example_files\pretrained_models\epoch=121-step=13053.ckpt")

# Evaluate the model on these step files.
evaluate_folder(step_folder, feature_standardization, pretrained_model)

0it [00:00, ?it/s]
GPU available: True, used: False


Completed pipeline/extract_feature_data_from_step.py


TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


 
Warning!! - No labels are provided.  This can happen when you
are evaluating with a pre-trained model on an unlabelled dataset.
Please disregard and accuracy and IoU values which get logged.
 


c:\Users\dnkmn\anaconda3\envs\brepnet\lib\site-packages\pytorch_lightning\trainer\trainer.py:1585: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."
c:\Users\dnkmn\anaconda3\envs\brepnet\lib\site-packages\pytorch_lightning\trainer\data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test/Chamfer_iou': 0.0,
 'test/CutEnd_iou': 1.0,
 'test/CutSide_iou': 0.0,
 'test/ExtrudeEnd_iou': 0.0,
 'test/ExtrudeSide_iou': 0.17142857611179352,
 'test/Fillet_iou': 0.0,
 'test/RevolveEnd_iou': 0.0,
 'test/RevolveSide_iou': 0.0,
 'test/accuracy': 0.17142857611179352,
 'test/mean_iou': 0.14642857015132904}
--------------------------------------------------------------------------------


In [4]:
logits_folder = Path("example_files/CustomStep/temp_working/logits")
step_file_stems = [ f.stem for f in step_folder.glob("*.stp")]
print(f"We found {len(step_file_stems)} example files")

example_index = 0
example_index1 = 1
example_index2 = 2
file_stem = step_file_stems[example_index]
print(f"Viewing example {file_stem}")
file_stem1 = step_file_stems[example_index1]
print(f"Viewing example {file_stem1}")
file_stem2 = step_file_stems[example_index2]
print(f"Viewing example {file_stem2}")
viewer = JupyterSegmentationViewer(file_stem, step_folder, seg_folder=step_folder, logit_folder=logits_folder)
viewer1 = JupyterSegmentationViewer(file_stem1, step_folder, seg_folder=step_folder, logit_folder=logits_folder)
viewer2 = JupyterSegmentationViewer(file_stem2, step_folder, seg_folder=step_folder, logit_folder=logits_folder)


We found 3 example files
Viewing example 001
Viewing example 002
Viewing example 003


Confirm the shape of your model:

In [5]:
viewer.view_solid()

In [6]:
viewer1.view_solid()

In [7]:
viewer2.view_solid()

Predict segmentation:

In [8]:
viewer.view_predicted_segmentation()

In [9]:
viewer1.view_predicted_segmentation()

In [10]:
viewer2.view_predicted_segmentation()

# Visualizing npz data
Once we have extracted BREP data from our STEP files - we can visualize their features in a few different ways.
The code blocks below explore different methods to visualize features of a BREP - Faces, Edges, Edge Direction, Coedges and Coedge Tagents.

In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import sys
print(sys.executable)
import os
if os.path.isfile("../models/brepnet.py"):
    os.chdir("../")
import tempfile
import numpy as np
    
import utils.data_utils as data_utils

# Imports from occwl
from occwl.io import load_step
from occwl.jupyter_viewer import JupyterViewer
from occwl.entity_mapper import EntityMapper


# Imports from BRepNet
from pipeline.extract_brepnet_data_from_step import extract_brepnet_data_from_step
import utils.scale_utils as scale_utils

c:\Users\dnkmn\anaconda3\envs\brepnet\python.exe


In [2]:
# Here is the path to some example step files for us to convert
step_folder = Path("./example_files/step_examples")
temp_folder = Path(tempfile.gettempdir())
working = temp_folder / "brepnet_test_working_dir"
if not working.exists():
    working.mkdir()

In [3]:
# Here we generate npz data from step files
extract_brepnet_data_from_step(step_folder, working)

100%|██████████| 25/25 [03:29<00:00,  8.38s/it]

Completed pipeline/extract_feature_data_from_step.py


In [4]:
all_npz_files = [ f for f in working.glob("*.npz") ]
npz_files = []
step_files = []
for npz_file in all_npz_files:
    step_file = step_folder / (npz_file.stem + ".stp")
    if step_file.exists():
        npz_files.append(npz_file)
        step_files.append(step_file)
    
print(f"Number of step and npz files {len(npz_files)}")

file_index = 0
npz_file = npz_files[file_index]
step_file = step_folder / (npz_file.stem + ".stp")
print("Viewing file") 
print(f"npz data {npz_file}")
print(f"STEP data {step_file}")

# Load the solid
solids = load_step(step_file)
print(f"Loaded {len(solids)} solids")
solid = solids[0]

# Scale to [-1,1]^3 box
solid = scale_utils.scale_solid_to_unit_box(solid)

# Load the npz data
data = data_utils.load_npz_data(npz_file)

Number of step and npz files 25
Viewing file
npz data C:\Users\dnkmn\AppData\Local\Temp\brepnet_test_working_dir\100155_57ec5fc6_0.npz
STEP data example_files\step_examples\100155_57ec5fc6_0.stp
Loaded 1 solids


In [6]:
viewer = JupyterViewer()
for face in solid.faces():
    viewer.display(face, render_edges=True)
viewer.show()

In [8]:
if len(viewer.selected_faces()) <= 0:
    print("Select some faces to view")
else:
    entity_mapper = EntityMapper(solid)
    selected_face_indices = viewer.selected_face_indices(entity_mapper)
    print(f"The faces you selected were {selected_face_indices}")

    # Get the point grids from the data
    face_grids = data["face_point_grids"]
    print(f"Shape of face grids {face_grids.shape}")
    selected_face_grids = face_grids[selected_face_indices]
    print(f"Shape of selected face grids {selected_face_grids.shape}")
    selected_face_grid_points = selected_face_grids[:,:3,:,:]
    points = np.transpose(np.reshape(selected_face_grid_points, (3, -1)))
    viewer.display_points(points)
    selected_face_grid_normals  = selected_face_grids[:,3:6,:,:]
    normals = np.transpose(np.reshape(selected_face_grid_normals, (3, -1)))
    viewer.display_unit_vectors(points, normals)

The faces you selected were [39 35 36]
Shape of face grids (44, 7, 10, 10)
Shape of selected face grids (3, 7, 10, 10)


# Edge reverse flag as color


In [9]:
edge_color_viewer = JupyterViewer()
edge_color_viewer.display(solid, render_edges=False)
for edge in solid.edges():
    if edge.reversed():
        color = "red"
    else:
        color = "blue"
    edge_color_viewer.display(edge, edge_color=color)
edge_color_viewer.show() 

# Edge reverse flag as direction

In [11]:
def display_dir_across_edge(viewer, coedge_grid):
    points = np.transpose(coedge_grid[:3, :])
    tangents = np.transpose(coedge_grid[3:6, :])
    left_normals = np.transpose(coedge_grid[6:9, :])
    across_edge_dir = np.cross(tangents, left_normals)
    viewer.display_points(points)
    viewer.display_unit_vectors(points, across_edge_dir, line_color="blue")

    
edge_dir_viewer = JupyterViewer()
edge_dir_viewer.display(solid, render_edges=True)
entity_mapper = EntityMapper(solid)
coedge_grids = data["coedge_point_grids"]
for edge in solid.edges():
    if edge.reversed():
        coedge = edge.reversed_edge()
    else:
        coedge = edge
    coedge_index = entity_mapper.oriented_edge_index(coedge)
    coedge_grid = coedge_grids[coedge_index]
    display_dir_across_edge(edge_dir_viewer, coedge_grid)
    
edge_dir_viewer.show()

# Coedge grids

In [12]:
coedge_viewer = JupyterViewer()
for face in solid.faces():
    coedge_viewer.display(face, render_edges=True)
coedge_viewer.show()

Select two adjacent faces so that we can visualize the coedge grid for the common edge.  The coedge is on the side of the first face.

In [87]:
def display_coedge_grid(viewer, grid):
    points = np.transpose(grid[:3, :])
    tangents = np.transpose(grid[3:6, :])
    left_normals = np.transpose(grid[6:9, :])
    right_normals = np.transpose(grid[9:12, :])
    viewer.display_points(points)
    viewer.display_unit_vectors(points, tangents, line_color="red")
    viewer.display_unit_vectors(points, left_normals, line_color="blue")
    viewer.display_unit_vectors(points, right_normals, line_color="green")
    
def find_selected_coedge(viewer, entity_mapper):
    selected_faces = viewer.selected_faces()
    for index, selected_face in enumerate(selected_faces):
        selected_face_index = entity_mapper.face_index(selected_face)
        print(f"Face {index}: {selected_face_index}")
        
    selected_faces_set = set(selected_faces)
       
    # Find the coedge to the left of the face 0 and right of face 1
    target_coedge = None
    for coedge in solid.edges():
        ffe = set(solid.faces_from_edge(coedge))
        if ffe == selected_faces_set:
            print("Found edges which shares faces")
            mate = coedge.reversed_edge()
            if len(selected_faces) == 2:
                if selected_faces[0].is_left_of(coedge) and selected_faces[1].is_left_of(mate):
                    target_coedge = coedge
                elif selected_faces[0].is_left_of(mate) and selected_faces[1].is_left_of(coedge):
                    target_coedge = mate
            elif len(selected_faces) == 1:
                assert False, "Need to implement"
    return target_coedge
    
num_selected_faces = len(coedge_viewer.selected_faces())
print(f"Found {num_selected_faces} selected faces")
if num_selected_faces > 0:
    entity_mapper = EntityMapper(solid)
    target_coedge = find_selected_coedge(coedge_viewer, entity_mapper)
            
    if target_coedge is not None:
        coedge_index = entity_mapper.oriented_edge_index(target_coedge)
        print(f"Coedge index {coedge_index}")
        coedge_grids = data["coedge_point_grids"]
        selected_coedge_grids = coedge_grids[coedge_index]
        display_coedge_grid(coedge_viewer, selected_coedge_grids)
    else:
        print("Found no coedge")
else:
    print("Please select one face and one edge")

Found 2 selected faces
Face 0: 1
Face 1: 39
Found edges which shares faces
Coedge index 10


# Coedge LCS matrix

In [88]:
lcs_viewer = JupyterViewer()
for face in solid.faces():
    lcs_viewer.display(face, render_edges=True)
lcs_viewer.show()

In [89]:
def display_coedge_lcs(viewer, selected_coedge_lcs):
    origin = np.expand_dims(selected_coedge_lcs[:, 3][:3], axis=0)
    u_vec = np.expand_dims(selected_coedge_lcs[:, 0][:3], axis=0)
    v_vec = np.expand_dims(selected_coedge_lcs[:, 1][:3], axis=0)
    w_vec = np.expand_dims(selected_coedge_lcs[:, 2][:3], axis=0)
    length = 0.02
    u_end = origin + u_vec*length
    v_end = origin + v_vec*length
    w_end = origin + w_vec*length
    viewer.display_points(origin)
    viewer.display_lines(origin, u_end, line_width=2, line_color="red")
    viewer.display_lines(origin, v_end, line_width=2, line_color="green")
    viewer.display_lines(origin, w_end, line_width=2, line_color="blue")

num_selected_faces = len(lcs_viewer.selected_faces())
print(f"Found {num_selected_faces} selected faces")
if num_selected_faces > 0:
    entity_mapper = EntityMapper(solid)
    target_coedge = find_selected_coedge(lcs_viewer, entity_mapper)
            
    if target_coedge is not None:
        coedge_index = entity_mapper.oriented_edge_index(target_coedge)
        print(f"Coedge index {coedge_index}")
        coedge_lcs = data["coedge_lcs"]
        selected_coedge_lcs = coedge_lcs[coedge_index]
        display_coedge_lcs(lcs_viewer, selected_coedge_lcs)
    else:
        print("Found no coedge")
else:
    print("Please select one face and one edge")

Found 2 selected faces
Face 0: 34
Face 1: 1
Found edges which shares faces
Coedge index 174


# View edge grids which UV-Net sees

In [91]:
uv_net_edge_viewer = JupyterViewer()
uv_net_edge_viewer.display(solid, render_edges=True)
entity_mapper = EntityMapper(solid)
coedge_grids = data["coedge_point_grids"]
for edge in solid.edges():
    coedge_index = entity_mapper.oriented_edge_index(edge)
    coedge_grid = coedge_grids[coedge_index]
    display_coedge_grid(uv_net_edge_viewer, coedge_grid)
uv_net_edge_viewer.show()

# Edge tangent directions

In [92]:
def display_edge_tangents(viewer, grid):
    points = np.transpose(grid[:3, [0, 2, 4, 6, 8]])
    
    view_dir = np.array([1,1,-1])
    
    # Find a good length for the arrows
    mins = np.min(points, axis=0)
    maxs = np.max(points, axis=0)
    diag = maxs - mins
    longest = np.max(diag)
    arrow_length = longest/8   
    arrow_head = arrow_length/4
    
    points += view_dir*arrow_head
    
    tangents = np.transpose(grid[3:6, [0, 2, 4, 6, 8]])
    arrow_ends = points + arrow_length*tangents
    arrow_dir_cross = view_dir/np.linalg.norm(view_dir)
    arrow_offset1 = np.cross(tangents, arrow_dir_cross)
    arrow_offset2 = -arrow_offset1
    arrow1 = arrow_ends - (tangents+arrow_offset1)*arrow_head
    arrow2 = arrow_ends - (tangents+arrow_offset2)*arrow_head
    line_width = 2
    viewer.display_lines(points, arrow_ends, line_color="blue", line_width=line_width)
    viewer.display_lines(arrow1, arrow_ends, line_color="blue", line_width=line_width)
    viewer.display_lines(arrow2, arrow_ends, line_color="blue", line_width=line_width)
    
edge_tangent_viewer = JupyterViewer()
edge_tangent_viewer.display(solid, render_edges=True)
entity_mapper = EntityMapper(solid)
coedge_grids = data["coedge_point_grids"]
for edge in solid.edges():
    if edge.reversed():
        # If the edge had the reverse flag set then the code in 
        # the EdgeDataExtractor already reversed the tangent directions.
        # To get the same tangent directions as the edges 3d curve
        # we need to get the coedge grid from the mate
        coedge = edge.reversed_edge()
    else:
        coedge = edge
    coedge_index = entity_mapper.oriented_edge_index(coedge)
    coedge_grid = coedge_grids[coedge_index]
    display_edge_tangents(edge_tangent_viewer, coedge_grid)
edge_tangent_viewer.show()

# Now that we understand how BRepNet reads and analyzes STEP files - lets explore how BRepNet can be used to find similarities between STEP files 

## Using the BRepNet input features as a face and edge similarity metric
BRepNet input features can be used as a similarity metric for finding similar faces

In [11]:
%load_ext autoreload
%autoreload 2

import json
import numpy as np
import os
if os.path.isfile('../models/brepnet.py'):
    # We are in the notebooks directory.  Change to the root
    os.chdir('../')

In [12]:
from pathlib import Path

# This viewer allows you to visualize the results
from visualization.jupyter_segmentation_viewer import JupyterSegmentationViewer

# Useful functions for loading data
import utils.data_utils as data_utils

In [13]:
# Here is the path to some example step files for us to convert
# step_folder = Path("./example_files/CustomStep")
step_folder = Path("./example_files/step_examples")

# We will also need to know the feature standardization for the dataset used to train the model
# This is found in the dataset file created by pipeline/build_dataset_file.py or pipeline/quickstart.py
feature_standardization_path = Path("./example_files/feature_standardization/s2.0.0_step_all_features.json")
standardization_data = data_utils.load_json_data(feature_standardization_path)
feature_standardization = standardization_data["feature_standardization"]

# This function to standardize features is similar to what we do in the dataloader
def standardize_features(feature_tensor, stats):
    num_features = len(stats)
    assert feature_tensor.shape[1] == num_features
    means = np.zeros(num_features)
    sds = np.zeros(num_features)
    eps = 1e-7
    for index, s in enumerate(stats):
        assert s["standard_deviation"] > eps, "Feature has zero standard deviation"
        means[index] = s["mean"]
        sds[index] = s["standard_deviation"]

    # We need to broadcast means and sds over the number of entities
    means_x = np.expand_dims(means, axis=0)
    sds_x = np.expand_dims(sds, axis=0)
    feature_tensor_zero_mean = feature_tensor - means_x
    feature_tensor_standadized = feature_tensor_zero_mean / sds_x

    # Convert the tensors to floats after standardization 
    return feature_tensor_standadized
    
def standarize_data(data, feature_standardization):
    data["face_features"] = standardize_features(data["face_features"], feature_standardization["face_features"])
    data["edge_features"] = standardize_features(data["edge_features"], feature_standardization["edge_features"])
    data["coedge_features"] = standardize_features(data["coedge_features"], feature_standardization["coedge_features"])
    return data
    
def find_faces_to_edges(coedge_to_face, coedge_to_edge):
    faces_to_edges_dict = {}
    for coedge_index in range(coedge_to_face.shape[0]):
        edge = coedge_to_edge[coedge_index]
        face = coedge_to_face[coedge_index]
        if not face in faces_to_edges_dict:
            faces_to_edges_dict[face] = set()
        faces_to_edges_dict[face].add(edge)
    faces_to_edges = []
    for i in range(len(faces_to_edges_dict)):
        assert i in faces_to_edges_dict
        faces_to_edges.append(faces_to_edges_dict[i])
    return faces_to_edges

# This function pools edges features onto all faces which
# are adjacent to that edge
def pool_edge_data_onto_faces(data):
    face_features = data["face_features"]
    edge_features = data["edge_features"]
    coedge_to_face = data["coedge_to_face"] 
    coedge_to_edge = data["coedge_to_edge"]
    for edge in coedge_to_edge:
        assert edge < edge_features.shape[0]
    faces_to_edges = find_faces_to_edges(coedge_to_face, coedge_to_edge)
    face_edge_features = []
    for face_edge_set in faces_to_edges:
        edge_features_for_face  = []
        for edge in face_edge_set:
            edge_features_for_face.append(edge_features[edge])
        pooled_edge_features = np.max(np.stack(edge_features_for_face), axis = 0)
        face_edge_features.append(pooled_edge_features)
    assert len(face_edge_features) == face_features.shape[0]
    face_edge_features = np.stack(face_edge_features)
    return np.concatenate([face_features, face_edge_features], axis = 1)

Choose the model you would like to use as the query

In [14]:
step_file_stems = [ f.stem for f in step_folder.glob("*.stp")]
print(f"We found {len(step_file_stems)} example files")

We found 25 example files


In [15]:
query_index = 12

Select some faces on the solid to continue.  
Double click on each face to select it

In [16]:
file_stem = step_file_stems[query_index]
print(f"Viewing example {file_stem}")
viewer = JupyterSegmentationViewer(file_stem, step_folder, seg_folder=step_folder)
viewer.view_solid()

Viewing example 24051_4852a192_5


In [7]:
assert len(viewer.selection_list) > 0, "Please select some faces on the solid"

In [8]:
# Load raw feature data
npz_folder = step_folder / "temp_working"
npz_pathname = npz_folder / (file_stem + ".npz")
data = data_utils.load_npz_data(npz_pathname)
data = standarize_data(data, feature_standardization)
face_features = data["face_features"]
edge_features = data["edge_features"]
pooled_face_edge_features = pool_edge_data_onto_faces(data)

assert pooled_face_edge_features.shape[0] == len(viewer.entity_mapper.face_map), "Embedding size doesn't match solid"
selected_faces_features = pooled_face_edge_features[viewer.selection_list]

In [9]:
all_files = list(npz_folder.glob("*.npz"))

min_dists_for_each_face = []
sum_min_dists_for_each_solid = []
for npz_file in all_files:
        
    # Load the input features
    data = data_utils.load_npz_data(npz_file)
    data = standarize_data(data, feature_standardization)
    pooled_face_edge_features = pool_edge_data_onto_faces(data)
        
    min_dists = []
    min_dists_for_each_face_in_this_solid = []
    for selected_face_features in selected_faces_features:
        vec = pooled_face_edge_features-selected_face_features
        dist = np.linalg.norm(vec, axis=1)
        
        # Here we compute the distance from a given query face
        # to each face in the solid
        min_dists_for_each_face_in_this_solid.append(dist)
        
        # Then is metric is the distance from that given query face to
        # the most similar face in the solid
        min_dists.append(np.min(dist))
    
    # Now we need some way to say which solids is the best match over all
    # We do this by summing the minimum distances to all the faces 
    sum_dists = np.sum(np.stack(min_dists), axis=0)
    sum_min_dists_for_each_solid.append(sum_dists)
    
    # For display we also keep track of the min distance to each face
    min_dists_for_each_face_in_this_solid = np.min(np.stack(min_dists_for_each_face_in_this_solid), axis=0)
    min_dists_for_each_face.append(min_dists_for_each_face_in_this_solid)
    
sum_min_dists_for_each_solid = np.array(sum_min_dists_for_each_solid)  

In [10]:
# Now we will search for the top k best matches
k = 5
indices_of_smallest = np.argpartition(sum_min_dists_for_each_solid, kth=range(k))[:k]

# We want to show how close each face in each solid is
# To do this we want some kind of range which is computed here
all_dists_top_k = []
for index in indices_of_smallest:
    all_dists_top_k.append(min_dists_for_each_face[index])
all_dists_top_k = np.concatenate(all_dists_top_k)
interval = [all_dists_top_k.min(), all_dists_top_k.max()]
print(f"Interval {interval}")

Interval [0.0, 7.467671834088819]


In [11]:
# Now as a sanity check we always expect to find the query as the best 
# match.  The other matching faces (red) should have a similar shape
for i, index in enumerate(indices_of_smallest):
    print(index)
    close_file_stem = all_files[index].stem
    print(f"Close file {close_file_stem}")
    close_viewer = JupyterSegmentationViewer(close_file_stem, step_folder)
    dists_to_view = min_dists_for_each_face[index]
    close_viewer.display_faces_with_heatmap(dists_to_view, interval)

12
Close file 24051_4852a192_5


24
Close file 97826_238f01e7_2


10
Close file 21492_8bd34fc1_15


11
Close file 24013_13d47495_3


16
Close file 42811_d1f01a68_2


## Using BRepNet embeddings for face similarity matching
We can use the embeddings generated by the BRepNet model can be used for selecitng similar faces.

In [1]:
# %load_ext autoreload
# %autoreload 2

import json
import numpy as np
import os
if os.path.isfile('../models/brepnet.py'):
    # We are in the notebooks directory.  Change to the root
    os.chdir('../')

In [2]:
from pathlib import Path

# This code allows you to evaluate a pre-trained model for all step files in a folder
from eval.evaluate_folder import evaluate_folder

# This viewer allows you to visualize the results
from visualization.jupyter_segmentation_viewer import JupyterSegmentationViewer

from visualization.save_images_of_similar_solids import SimilarSolidImageSaver
from occwl.solid import Solid

To evaluate a collection of STEP files we need to use `eval/evaluate_folder.py`.  

Provide the the script with the path to the step files to evaluate, the feature standadization and the pretrained model.


In [3]:
# Add the path to the dataset folder containng the step files:
# step_folder = Path("./example_files/CustomStep")
step_folder = Path("./example_files/step_examples")



# You can run this on the entire extended STEP dataset which you can doanload from 
# https://fusion-360-gallery-dataset.s3.us-west-2.amazonaws.com/segmentation/s2.0.0/s2.0.0_extended_step.zip
# step_folder = Path("/path/to/s2.0.0_extended_step/breps/step/")

# Provide the feature standardization for the dataset used to train the model
feature_standardization = Path("./example_files/feature_standardization/s2.0.0_step_all_features.json")

# Provide a pretrained model
pretrained_model = Path("./example_files/pretrained_models/pretrained_s2.0.0_extended_step_uv_net_features_0816_183419.ckpt")

# This function evaluates the model on your step files.
evaluate_folder(step_folder, feature_standardization, model=pretrained_model)

0it [00:00, ?it/s]
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Completed pipeline/extract_feature_data_from_step.py


c:\Users\dnkmn\anaconda3\envs\brepnet\lib\site-packages\pytorch_lightning\trainer\trainer.py:1585: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."


Using labels from example_files\step_examples\temp_working


c:\Users\dnkmn\anaconda3\envs\brepnet\lib\site-packages\pytorch_lightning\trainer\data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test/Chamfer_iou': 0.8418079018592834,
 'test/CutEnd_iou': 0.7290322780609131,
 'test/CutSide_iou': 0.8102856874465942,
 'test/ExtrudeEnd_iou': 0.7076271176338196,
 'test/ExtrudeSide_iou': 0.8099502325057983,
 'test/Fillet_iou': 0.9281437397003174,
 'test/RevolveEnd_iou': 0.8999999761581421,
 'test/RevolveSide_iou': 0.7386363744735718,
 'test/accuracy': 0.8996027708053589,
 'test/mean_iou': 0.8081854581832886}
--------------------------------------------------------------------------------


In [4]:
step_file_stems = [ f.stem for f in step_folder.glob("*.stp")]
print(f"We found {len(step_file_stems)} example files")

We found 25 example files


Select the model that you want to use as a base reference for your query:

In [5]:
query_index = 10

Select faces on the solid before proceeding to the next step (Double click on a face to select it)


In [6]:
file_stem = step_file_stems[query_index]
print(f"Viewing example {file_stem}")
viewer = JupyterSegmentationViewer(file_stem, step_folder, seg_folder=step_folder)
viewer.view_solid()

Viewing example 21492_8bd34fc1_15


In [7]:
assert len(viewer.selection_list) > 0, "Please select some faces on the solid"

Load the embeddings for the query solid

In [8]:
# Load the embeddings for the query solid
embeddings_folder = step_folder / "temp_working/embeddings"
embeddings_pathname = embeddings_folder / (file_stem + ".embeddings")
embeddings = np.loadtxt(embeddings_pathname)
assert embeddings.shape[0] == len(viewer.entity_mapper.face_map), "Embedding size doesn't match solid"
selected_face_embeddings = embeddings[viewer.selection_list]

In [9]:
# Loop over all solids and find the distance from each face of each solid to each query face
all_embeddings = list(embeddings_folder.glob("*.embeddings"))
min_dists_for_each_face = []
sum_min_dists_for_each_solid = []
for embedding_file in all_embeddings:
    target_file_stem = embedding_file.stem
    
    # Load the embeddings for the target file
    embeddings = np.loadtxt(embedding_file)
    if len(embeddings.shape) == 1:
        embeddings = np.expand_dims(embeddings, axis=0)
        
    min_dists = []
    min_dists_for_each_face_in_this_solid = []
    for face_embedding in selected_face_embeddings:
        face_to_embeddings = embeddings-face_embedding
        dist = np.linalg.norm(face_to_embeddings, axis=1)
        
        # Compute the distance from a given query face to each face in the solid
        min_dists_for_each_face_in_this_solid.append(dist)
        
        # A metric is generated that represents the distance from a given query face to the most similar face in the solid
        min_dists.append(np.min(dist))
    
    # To say which solids is the best match overall - we sum the minimum distances to all the faces 
    sum_dists = np.sum(np.stack(min_dists), axis=0)
    sum_min_dists_for_each_solid.append(sum_dists)
    
    # For display purposes, we keep track of the min distance to each face
    min_dists_for_each_face_in_this_solid = np.min(np.stack(min_dists_for_each_face_in_this_solid), axis=0)
    min_dists_for_each_face.append(min_dists_for_each_face_in_this_solid)
    
sum_min_dists_for_each_solid = np.array(sum_min_dists_for_each_solid)      

In [10]:
# Now we will search for the top k best matches
k = 4
indices_of_smallest = np.argpartition(sum_min_dists_for_each_solid, kth=range(k))[:k]

# We want to show how close each face in each solid is we define a range.
all_dists_top_k = []
for index in indices_of_smallest:
    all_dists_top_k.append(min_dists_for_each_face[index])
all_dists_top_k = np.concatenate(all_dists_top_k)
interval = [all_dists_top_k.min(), all_dists_top_k.max()]
print(f"Interval {interval}")

Interval [0.0, 19.19418612741993]


In [11]:
# As a sanity check we always expect to find the query as the best match. All other matching faces (red) should have a similar shape
for i, index in enumerate(indices_of_smallest):
    print(index)
    close_file_stem = all_embeddings[index].stem
    print(f"Close file {close_file_stem}")
    close_viewer = JupyterSegmentationViewer(close_file_stem, step_folder)
    dists_to_view = min_dists_for_each_face[index]
    close_viewer.display_faces_with_heatmap(dists_to_view, interval)

10
Close file 21492_8bd34fc1_15


18
Close file 44647_d83249a9_0


24
Close file 97826_238f01e7_2


22
Close file 56436_2a8fc254_3
